# Save extended QC for bwm dataset

In [1]:
"""
Check sessions available with dlc data
@author: Ines
"""
import pandas as pd
import pickle 
import csv
import seaborn as sns
import matplotlib.pyplot as plt
import os
import numpy as np
from one.api import ONE
from uuid import UUID

# Get my functions
functions_path =  '/home/ines/repositories/representation_learning_variability/Functions/'
#functions_path = '/Users/ineslaranjeira/Documents/Repositories/representation_learning_variability/Functions/'

os.chdir(functions_path)
from video_functions import get_lick_times, lick_psth
from data_processing import save_pickle_and_log

one = ONE(mode='remote')

### Get QC of all sessions

In [2]:
data_path =  '/home/ines/repositories/representation_learning_variability/DATA/Brainwidemap/'
# data_path = '/Users/ineslaranjeira/Documents/Repositories/representation_learning_variability/DATA/Brainwidemap/'

os.chdir(data_path)
qc = pd.read_csv("Brainwidemap_QC - behavior_data.csv")


In [3]:
def extended_left_dlc(one, qc):
    
    eids = qc['Session ID']

    # Initialize df
    df = pd.DataFrame()

    for e, eid in enumerate(eids):
        
        extended_qc = one.get_details(eid, True)['extended_qc']
        transposed_df = pd.DataFrame.from_dict(extended_qc, orient='index').T
        transposed_df['eid'] = eid
        df = df.append(transposed_df)
    
    return df

In [4]:
ext_qc = extended_left_dlc(one, qc)

/tmp/ipykernel_2968870/541224780.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(transposed_df)
/tmp/ipykernel_2968870/541224780.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(transposed_df)
/tmp/ipykernel_2968870/541224780.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(transposed_df)
/tmp/ipykernel_2968870/541224780.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(transposed_df)
/tmp/ipykernel_2968870/541224780.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = 

In [9]:
import os
import json
from datetime import datetime
import pickle
from collections import defaultdict

def save_pickle_and_log(file_to_save, filename, save_path):

    # current date
    now = datetime.now() # current date and time
    date_time = now.strftime("%m-%d-%Y")
    
    # First, save file in desired location
    os.chdir(save_path)
    pickle.dump(file_to_save, open(filename+date_time, "wb"))
    
    # Open log file json
    log_file_path =  '/home/ines/repositories/representation_learning_variability/DATA/' 
    file = log_file_path + 'metadata_log.json'
    with open(file, 'r') as openfile:
        # Reading from json file
        metadata_log = json.load(openfile)
    
    # Then create entry for the log file
    files = [f for f in os.listdir() if f.endswith('.ipynb')]
    script_name = files[0]
    
    # Populate new entry
    new_log_entry = {
    "data_filename": filename,
    "script_name": script_name,
    "timestamp": date_time
    }
    
    # Update log dict
    order_last_entry = int(list(metadata_log.keys())[-1])
    metadata_log[order_last_entry+1] = new_log_entry
    
    # Overwrite json log file
    with open(log_file_path+"metadata_log.json", "w") as outfile:
        outfile.write(str(metadata_log))
        
    return metadata_log


In [11]:
file_to_save = ext_qc
filename = "dlc_left_extended_qc"
save_path = '/home/ines/repositories/representation_learning_variability/Video and wheel/Video QC/'

metadata = save_pickle_and_log(file_to_save, filename, save_path)

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)

In [ ]:
# save_path = '/home/ines/repositories/representation_learning_variability/Video and wheel/Video QC'
# os.chdir(save_path)
# pickle.dump(ext_qc, open("dlc_left_extended_qc", "wb"))

# Custom lick QC

In [ ]:
ext_qc_path = '/home/ines/repositories/representation_learning_variability/Video and wheel/Video QC/'
os.chdir(ext_qc_path)
ext_qc = pickle.load(open(ext_qc_path + "dlc_left_extended_qc", "rb"))
use_qc = ext_qc.loc[ext_qc['task'].isin(['PASS', 'WARNING'])]

# psth_df = pickle.load(open(ext_qc_path + "dlc_left_extended_psth", "rb"))
# lick_custom = pickle.load(open(ext_qc_path + "dlc_left_extended_lick", "rb"))


In [ ]:
def custom_qc(df):
    # Calculate licks per trial and append to extended qc df
    df['ratio'] = df['behavior'] * np.nan
    df['miss_lick_count'] = df['behavior'] * np.nan

    for e, eid in enumerate(df['eid'].unique()):
        try:
            lick_times = get_lick_times(one, eid, video_type = 'left')
            trials = one.load_object(eid, obj='trials', namespace='ibl')
            trials_df = trials.to_df()
            licks_psth_post = lick_psth(trials_df, lick_times, 0, 1, event='feedback_times')

            # Save
            df.loc[df['eid']==eid, 'ratio'] = round(len(lick_times) / len(trials_df), 2)
            if len(licks_psth_post) > 0:
                df.loc[df['eid']==eid, 'miss_lick_count'] = np.max(licks_psth_post['trial']) - len(licks_psth_post['trial'].unique())
            
        except:
            print('No licks for session '+str(eid))
            
    return df

In [ ]:
eids = use_qc['eid'].unique()
lick_custom = custom_qc(use_qc)

In [ ]:
save_path = '/home/ines/repositories/representation_learning_variability/Video and wheel/Video QC'
os.chdir(save_path)
pickle.dump(lick_custom, open("dlc_left_extended_lick", "wb"))

# Get specific values from extended QC

In [103]:

ext_qc_path = '/home/ines/repositories/representation_learning_variability/Video and wheel/Video QC/'
os.chdir(ext_qc_path)
ext_qc = pickle.load(open(ext_qc_path + "dlc_left_extended_qc", "rb"))
use_qc = ext_qc.loc[ext_qc['task'].isin(['PASS', 'WARNING'])]

psth_df = pickle.load(open(ext_qc_path + "dlc_left_extended_psth", "rb")) 
lick_custom = pickle.load(open(ext_qc_path + "dlc_left_extended_lick", "rb")) # Check extended_qc.ipynb code

# ON 16DEC2024 THIS IS THE CUSTOM QC I PLAN TO USE
def full_custom_qc(one, qc):
    
    # Extend QC 
    ext_qc = extended_left_dlc(one, qc)
    # Filter by task QC
    task_qc = ext_qc.loc[ext_qc['task'].isin(['PASS', 'WARNING'])]
    # Apply custom lick QC
    lick_custom = custom_qc(use_qc)
    # Filter based on custom lick qc and some key video qcs
    final_custom_qc = lick_custom.loc[(lick_custom['dlcLeft'].isin(['FAIL', 'WARNING', 'PASS', np.nan])) &
                    (lick_custom['videoLeft'].isin(['FAIL', 'WARNING', 'PASS', np.nan])) &
                    (lick_custom['ratio']>5) &
                    (lick_custom['_videoLeft_pin_state'].apply(lambda x: (isinstance(x, list) and True in x) or x == 'PASS')) &
                    (lick_custom['_videoLeft_framerate'].apply(lambda x: (isinstance(x, list) and True in x) or x == 'PASS')) &  
                    (lick_custom['_videoLeft_camera_times'].apply(lambda x: (isinstance(x, list) and True in x) or x == 'PASS')) &
                    (lick_custom['_videoLeft_dropped_frames'].apply(lambda x: (isinstance(x, list) and True in x) or x == 'PASS' or x == None)) &
                    (lick_custom['_videoLeft_timestamps'].isin([True, 'PASS', np.nan]))                   
                    ]
    
    return final_custom_qc

In [ ]:
use_qc = ext_qc.loc[ext_qc['task'].isin(['PASS', 'WARNING'])]

psth_df = pickle.load(open(ext_qc_path + "dlc_left_extended_psth", "rb"))
lick_custom = pickle.load(open(ext_qc_path + "dlc_left_extended_lick", "rb"))

use_filter = lick_custom.loc[(lick_custom['dlcLeft'].isin(['FAIL', 'WARNING', 'PASS', np.nan])) &
                             (lick_custom['videoLeft'].isin(['FAIL', 'WARNING', 'PASS', np.nan])) &
                             (lick_custom['ratio']>5) &
                             (lick_custom['_videoLeft_pin_state'][0]=='PASS')
                            ]
eids = use_filter['eid'].unique()
files = os.listdir(Path.home()/'Output LP')

videoLeft_qc = ['eid', 'videoLeft', '_videoLeft_position',  '_videoLeft_pin_state',
                '_videoLeft_framerate', '_videoLeft_resolution',
                '_videoLeft_brightness', '_videoLeft_timestamps',
                '_videoLeft_file_headers', '_videoLeft_camera_times',
                '_videoLeft_focus', '_videoLeft_dropped_frames',
                '_videoLeft_wheel_alignment' ]
dlcLeft_qc = ['eid', 'dlcLeft', '_dlcLeft_time_trace_length_match', '_dlcLeft_trace_all_nan',
              '_dlcLeft_mean_in_bbox', '_dlcLeft_pupil_blocked', 
              '_dlcLeft_lick_detection'
]


for e, eid in enumerate(eids):
    print(eid)
    for k in videoLeft_qc:
        a = use_filter.loc[use_filter['eid']==eid, k]
        print(k + str(list(a)))

In [ ]:
use_qc = ext_qc.loc[ext_qc['task'].isin(['PASS', 'WARNING'])]

psth_df = pickle.load(open(ext_qc_path + "dlc_left_extended_psth", "rb"))
lick_custom = pickle.load(open(ext_qc_path + "dlc_left_extended_lick", "rb"))

use_filter = lick_custom.loc[(lick_custom['dlcLeft'].isin(['FAIL', 'CRITICAL', np.nan])) &
                             (lick_custom['videoLeft'].isin(['FAIL', 'CRITICAL', np.nan])) &
                            (lick_custom['ratio']>5) &
                            (lick_custom['miss_lick_count']<100)]
eids = use_filter['eid'].unique()
files = os.listdir(Path.home()/'Output LP')

videoLeft_qc = ['eid', 'videoLeft', '_videoLeft_position',  '_videoLeft_pin_state',
                '_videoLeft_framerate', '_videoLeft_resolution',
                '_videoLeft_brightness', '_videoLeft_timestamps',
                '_videoLeft_file_headers', '_videoLeft_camera_times',
                '_videoLeft_focus', '_videoLeft_dropped_frames',
                '_videoLeft_wheel_alignment' ]
dlcLeft_qc = ['eid', 'dlcLeft', '_dlcLeft_time_trace_length_match', '_dlcLeft_trace_all_nan',
              '_dlcLeft_mean_in_bbox', '_dlcLeft_pupil_blocked', 
              '_dlcLeft_lick_detection'
]

check_eids = ['4b7fbad4-f6de-43b4-9b15-c7c7ef44db4b',
'9a629642-3a9c-42ed-b70a-532db0e86199',
'8c2f7f4d-7346-42a4-a715-4d37a5208535',
'8db36de1-8f17-4446-b527-b5d91909b45a',
'1eac875c-feaa-4a30-b148-059b954b11d8',
'41431f53-69fd-4e3b-80ce-ea62e03bf9c7',
'a66f1593-dafd-4982-9b66-f9554b6c86b5',
'a82800ce-f4e3-4464-9b80-4c3d6fade333',
'6a601cc5-7b79-4c75-b0e8-552246532f82',
'20c112a1-8a42-44e0-a4cd-e5b932f7bda9']
for e, eid in enumerate(eids):
    print(eid)
    for k in videoLeft_qc:
        a = use_filter.loc[use_filter['eid']==eid, k]
        print(k + str(list(a)))

In [ ]:
no_dlc_plot = ['4b7fbad4-f6de-43b4-9b15-c7c7ef44db4b',
'9a629642-3a9c-42ed-b70a-532db0e86199',
'8c2f7f4d-7346-42a4-a715-4d37a5208535',
'8db36de1-8f17-4446-b527-b5d91909b45a',
'1eac875c-feaa-4a30-b148-059b954b11d8',
'41431f53-69fd-4e3b-80ce-ea62e03bf9c7',
'a66f1593-dafd-4982-9b66-f9554b6c86b5',
'a82800ce-f4e3-4464-9b80-4c3d6fade333',
'6a601cc5-7b79-4c75-b0e8-552246532f82',
'20c112a1-8a42-44e0-a4cd-e5b932f7bda9']
no_whisker = ['4b7fbad4-f6de-43b4-9b15-c7c7ef44db4b',
              'cf43dbb1-6992-40ec-a5f9-e8e838d0f643',
'4a45c8ba-db6f-4f11-9403-56e06a33dfa4',
'10fac7a1-919d-4ca7-83c4-4675bd8d9416',
'493170a6-fd94-4ee4-884f-cc018c17eeb9',
'edd22318-216c-44ff-bc24-49ce8be78374',
'73918ae1-e4fd-4c18-b132-00cb555b1ad2',
'25d1920e-a2af-4b6c-9f2e-fc6c65576544',
'eebacd5a-7dcd-4ba6-9dff-ec2a4d2f19e0',
'a4a74102-2af5-45dc-9e41-ef7f5aed88be',
'493170a6-fd94-4ee4-884f-cc018c17eeb9'] 


In [ ]:
np.intersect1d(no_dlc_plot, no_whisker)

array(['4b7fbad4-f6de-43b4-9b15-c7c7ef44db4b'], dtype='<U36')

In [ ]:
'4b7fbad4-f6de-43b4-9b15-c7c7ef44db4b' # no dlc ; QC fails for time_trace_length_match - lick plot worked
'9a629642-3a9c-42ed-b70a-532db0e86199' # length does not match; QC fails for time_trace_length_match 
'8c2f7f4d-7346-42a4-a715-4d37a5208535' # length does not match; QC fails for time_trace_length_match 
'8db36de1-8f17-4446-b527-b5d91909b45a' # could not broadcast input array from shape (20,20,3) into shape (0,20,3); QC fails for time_trace_length_match + mean not in box
'1eac875c-feaa-4a30-b148-059b954b11d8' # could not broadcast input array from shape (20,20,3) into shape (0,20,3); QC fails for time_trace_length_match + mean not in box
'41431f53-69fd-4e3b-80ce-ea62e03bf9c7' # could not broadcast inp; QC fails for time_trace_length_match + mean not in box
'a66f1593-dafd-4982-9b66-f9554b6c86b5' # could not broadcast input array from shape (20,20,3) into shape (0,20,3); QC fails for time_trace_length_match + mean not in box
'a82800ce-f4e3-4464-9b80-4c3d6fade333' # length does not match; QC fails for time_trace_length_match + mean not in box
'6a601cc5-7b79-4c75-b0e8-552246532f82' # length does not match; QC fails for time_trace_length_match + mean not in box
'20c112a1-8a42-44e0-a4cd-e5b932f7bda9' # length does not match; QC fails for trace all nan